# About: single-userのコンテナイメージを変更する

---

生徒の利用環境となる single-user Jupyter Notebook serverのコンテナイメージを変更します。 

## UnitGroup名

操作対象となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/

操作対象となるUnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

構築対象となる各VCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

In [ ]:
target_nodes = f'{ugroup_name}_worker'

!ansible {target_nodes} -m ping

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

## コンテナイメージ

新たに設定する single-userコンテナのイメージ名を指定してください。

以下のようなものを指定できます。

* [niicloudoperation/notebook](https://github.com/NII-cloud-operation/Jupyter-LC_docker)
    - JupyterNotebookに*Literate Computing for Reproducible Infrastructure*のためのツールを追加したもの
* harbor.vcloud.nii.ac.jp/vcp/coursewarehub:singleuser-disable-run-through
    - `niicloudoperation/notebook`に対して[Jupyter-LC_run_through](https://github.com/NII-cloud-operation/Jupyter-LC_run_through)の機能などを無効にしたもの

In [ ]:
# (例)
# singleuser_image = 'niicloudoperation/notebook'
# singleuser_image = 'harbor.vcloud.nii.ac.jp/vcp/coursewarehub:singleuser-disable-run-through'

singleuser_image = 

## 現在の設定を確認

single-user serverとして設定されている現在のコンテナイメージのIDを確認します。

In [ ]:
!ansible {ugroup_name} -m shell -a \
    'docker images | grep niicloudoperation/jupyterhub-singleuser'

## コンテナイメージの変更

single-user serverのコンテナイメージを指定されたものに変更します。

各ノードで新しいコンテナイメージの取得を行います。

In [ ]:
!ansible {ugroup_name} -a 'docker pull {singleuser_image}'

取得したコンテナイメージにsingle-userのイメージとしてのタグを設定します。

In [ ]:
!ansible {ugroup_name} -a \
    'docker tag {singleuser_image} niicloudoperation/jupyterhub-singleuser'

`IMAGE ID`が新しいコンテナイメージに更新されたことを確認します。

In [ ]:
!ansible {ugroup_name} -m shell -a \
    'docker images | grep niicloudoperation/jupyterhub-singleuser'